# 연관분석 개요
- 데이터들 사이에서 '자주 발생하는 속성을 찾고, 그 속성들 사이에 어느 정도 있는지를 분석하는 방법
- 활용분야 : 상품진열, 사기보험 적발, 카탈로그 디자인, 신상품 카테고리 구성 
- 연관성 분석 관련 지표 :
    지지도(support): 조건결과항목수/전체수
    신뢰도(confidense): 조건결과항목수/조건항목수 
    향상도(lift): 우연히 발생한 규칙인지 아닌지. 조건결과지지도/(조건지지도x결과지지도).
                  향상도가 1이면 상관관계 없음으로 봄. 유의미한 결과가 아니다.
                  1보다 크면 양의 상관관계.
                  1보다 작으면 음의 상관관계.
    [조건]=>[결과]  지지도 | 신뢰도 | 향상도
    [주스]=>[생수]   0.4        1      0.4/(0.4)x1 = 1
    [소주]=>[맥주]   0.2      0.333    0.2/(0.6x0.4)
  
소주, 생수, 와인
소주, 오렌지주스, 생수
생수, 맥주, 와인
소주, 생수, 맥주
오렌지주스, 와인, 생수

In [1]:
import os
os.getcwd()

'E:\\kim_jiwon\\src\\09_자연어처리'

In [5]:
# 트랜젝션 데이터 가져오기
import csv
transaction = []
with open('cf_basket.csv','r',encoding='utf-8') as cf:
    csvdata = csv.reader(cf)
    #print(list(csvdata))
    #transaction = list(csvdata)
    for row in csvdata:
        transaction.append(row)
transaction

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['콜라', '맥주', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인']]

# 2. 연관분석
- pip install apyori

In [48]:
from apyori import apriori
rules = apriori(transaction, min_support = 0.2, min_confidence=0.1)
rules=list(rules)
len(rules)

18

In [49]:
rules[1]

RelationRecord(items=frozenset({'소주'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'소주'}), confidence=0.6, lift=1.0)])

In [50]:
print('조건-> 결과 \t 지지도 \t 신뢰도 \t 향상도')
for row in rules:
    support=row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ', '.join(x for x in item[0])
        lhs = [x for x in item[0]]
        lhs = ', '.join(x for x in item[1])
        rhs = [x for x in item[1]]
        confidence = item[2]
        lift = item[3]
        if lift !=1:
            print(lhs,rhs,'\t{5.3f},\t{5.3f},\t{5.3f}'.format(support,confidence,lift))

조건-> 결과 	 지지도 	 신뢰도 	 향상도


IndexError: Replacement index 5 out of range for positional args tuple

In [51]:

print('조건   ->    결과 \t 지지도 \t 신뢰도 \t 향상도')
for row in rules:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ', '.join(x for x in item[0])
        lhs = [x for x in item[0]]
        rhs = ', '.join(x for x in item[1])
        rhs = [x for x in item[1]]
        confidence = item[2]
        lift = item[3]
        if lift != 1:
            print(lhs, '=>', rhs, "\t{:5.3f}\t{:5.3f}\t{:5.3f}".format(support,
                                                confidence, lift))

조건   ->    결과 	 지지도 	 신뢰도 	 향상도
['맥주'] => ['소주'] 	0.200	0.500	0.833
['소주'] => ['맥주'] 	0.200	0.333	0.833
['맥주'] => ['와인'] 	0.200	0.500	0.833
['와인'] => ['맥주'] 	0.200	0.333	0.833
['맥주'] => ['콜라'] 	0.400	1.000	1.250
['콜라'] => ['맥주'] 	0.400	0.500	1.250
['소주'] => ['오렌지주스'] 	0.200	0.333	0.833
['오렌지주스'] => ['소주'] 	0.200	0.500	0.833
['소주'] => ['와인'] 	0.200	0.333	0.556
['와인'] => ['소주'] 	0.200	0.333	0.556
['소주'] => ['콜라'] 	0.600	1.000	1.250
['콜라'] => ['소주'] 	0.600	0.750	1.250
['오렌지주스'] => ['와인'] 	0.200	0.500	0.833
['와인'] => ['오렌지주스'] 	0.200	0.333	0.833
['오렌지주스'] => ['콜라'] 	0.200	0.500	0.625
['콜라'] => ['오렌지주스'] 	0.200	0.250	0.625
['와인'] => ['콜라'] 	0.400	0.667	0.833
['콜라'] => ['와인'] 	0.400	0.500	0.833
['맥주'] => ['소주', '콜라'] 	0.200	0.500	0.833
['소주'] => ['콜라', '맥주'] 	0.200	0.333	0.833
['콜라'] => ['소주', '맥주'] 	0.200	0.250	1.250
['소주', '맥주'] => ['콜라'] 	0.200	1.000	1.250
['콜라', '맥주'] => ['소주'] 	0.200	0.500	0.833
['소주', '콜라'] => ['맥주'] 	0.200	0.333	0.833
['맥주'] => ['콜라', '와인'] 	0.200	0.500	1.250
['와인'] =

In [ ]:
# 뉴스기사 연관분석 실습

## 3.1 뉴스 RSS 이용해서 기사 검색 후 연관분석 

In [34]:
import requests
from bs4 import BeautifulSoup
rss_url = 'https://rss.joins.com/joins_money_list.xml'
money_response = requests.get(rss_url)
money_soup = BeautifulSoup(money_response.content, "xml")
link_list = money_soup.select("item > link")
link_list=[l.text for l in link_list]
len(link_list)

30

In [42]:
from konlpy.tag import Kkma
kkma = Kkma()
news=[]
for link in link_list:
    news_response =requests.get(link)
    news_soup = BeautifulSoup(news_response.content,"html.parser")
    content = news_soup.select_one('div#article_body').text
    nouns = list(filter(lambda word : len(word)>1, kkma.nouns(content)))
    news.append(nouns)


In [43]:
news

[['26',
  '26일',
  '승진',
  '대상',
  '대상그룹',
  '그룹',
  '부회장',
  '회장',
  '사진',
  '44',
  '전무',
  '지주',
  '지주회사',
  '회사',
  '대상홀딩스와',
  '홀딩',
  '스와',
  '임창',
  '명예',
  '명예회장',
  '장녀',
  '차녀',
  '임상',
  '임상민',
  '41',
  '자매',
  '경영',
  '체제',
  '본격',
  '궤도',
  '해석',
  '부회',
  '대상홀딩스',
  '딩스',
  '전략',
  '전략담당',
  '담당',
  '중역',
  '기존',
  '마케팅',
  '마케팅담당',
  '보직',
  '동시',
  '수행',
  '관련',
  '대상홀딩',
  '이날',
  '정기',
  '주주',
  '주주총회',
  '총회',
  '사내',
  '사내이사',
  '이사',
  '신규',
  '선임',
  '안건',
  '의결',
  '연세대',
  '경영학',
  '공부',
  '이후',
  '미국',
  '뉴욕',
  '뉴욕대',
  '심리학',
  '전공',
  '1998',
  '1998년',
  '이재',
  '삼성',
  '삼성전자',
  '전자',
  '결혼',
  '2009',
  '2009년',
  '이혼',
  '2012',
  '2012년',
  '크리',
  '티브',
  '디렉터',
  '직책',
  '합류',
  '2014',
  '2014년',
  '청정원',
  '정원',
  '대규모',
  '리뉴얼',
  '주도',
  '주도해',
  '브랜드',
  '이미지',
  '강화',
  '2016',
  '2016년',
  '안주야',
  '주야',
  '출시',
  '국내',
  '안주',
  '가정',
  '가정간편식',
  '편식',
  '시장',
  '개척',
  '설명',
  '관계자',
  '변화',
  '정확',
  '정보',
  '습득',
  '실행',
  '차원',
  '중장기',


In [44]:
from apyori import apriori
rules = apriori(news,min_support=0.3, min_confidence=0.2)
result=list(rules)

In [45]:
for r in result:
    print(r, end='\n')

RelationRecord(items=frozenset({'10'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'10'}), confidence=0.3333333333333333, lift=1.0)])
RelationRecord(items=frozenset({'19'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'19'}), confidence=0.3333333333333333, lift=1.0)])
RelationRecord(items=frozenset({'26'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'26'}), confidence=0.6, lift=1.0)])
RelationRecord(items=frozenset({'26일'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'26일'}), confidence=0.6, lift=1.0)])
RelationRecord(items=frozenset({'결과'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'결과'}), confidence=0.3, lift=1.0)])
RelationRecord(items=frozenset({'계획'}), support=0.36666666666666664, ordered_statis

In [52]:
import pandas as pd
result_df =  pd.DataFrame(None, 
                    columns=['lhs','rhs','support','confidence','lift'])
index = 0
for row in result:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ','.join(x for x in item[0])
        rhs = ','.join(x.strip() for x in item[1])
        confidence = item[2]
        lift = item[3]
        if lift != 1:
            result_df.loc[index]=[lhs, rhs, support, confidence, lift]
            index += 1
result_df

,lhs,rhs,support,confidence,lift
0,19,기자,0.300000,0.900000,1.173913
1,기자,19,0.300000,0.391304,1.173913
2,26,26일,0.600000,1.000000,1.666667
3,26일,26,0.600000,1.000000,1.666667
4,26,관련,0.433333,0.722222,0.984848
...,...,...,...,...,...
729,"주주총회,그룹,이사,주주",총회,0.300000,1.000000,3.333333
730,"그룹,이사,총회,주주",주주총회,0.300000,1.000000,3.333333
731,"주주총회,그룹,이사,총회",주주,0.300000,1.000000,3.000000
732,"주주총회,그룹,총회,주주",이사,0.300000,1.000000,3.333333


In [53]:
result_df.loc[(result_df.lhs.str.contains("기자"))&
             result_df.rhs.str.contains("한국")].sort_values(by=['lift'],ascending=False)

,lhs,rhs,support,confidence,lift
329,"관련,기자",한국,0.333333,0.555556,1.111111
327,기자,"관련,한국",0.333333,0.434783,1.086957
112,기자,한국,0.366667,0.478261,0.956522


In [ ]:
pd.options.display.max_rows = 644
result_df

In [ ]:
result_df.loc[(result_df.lhs.str.contains("기자"))].sort_values(by=['lift'],
                                                             ascending=False)

In [ ]:
result_df.loc[(result_df.lhs.str.contains("기자")) &
             result_df.rhs.str.contains("한국")].sort_values(by=['lift'],
                                                             ascending=False)

In [ ]:
# 장난삼아 연습
import requests
from bs4 import BeautifulSoup
from konlpy.tag import Kkma
kkma = Kkma()
test_url = "https://search.daum.net/search?nil_suggest=btn&w=tot&DA=SBC&q=%EC%BD%94%EB%A1%9C%EB%82%98"
test_response = requests.get(test_url)
test_response
test_soup = BeautifulSoup(test_response.content, "html.parser")
text_content = test_soup.select('.f_eb')
news = []
for text in text_content:
        news.append(list(filter(lambda word : len(word)>1 and word!='기자',
                               kkma.nouns(text.text))))

In [ ]:
news_nouns = set()
news
for new in news:
    for n in new:
        news_nouns.add(n)
print('코로나 기사에 나오는 단어들 :',news_nouns)
print(len(news_nouns))